# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [20]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath
from azureml.data.dataset_factory import TabularDatasetFactory

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mobile_price_prediction'

experiment=Experiment(ws, experiment_name)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

Cluster already exists


In [7]:
input_data = pd.read_csv("https://raw.githubusercontent.com/valeriapolozun/machine_learning_azure_capstone/803e04124532b8921a0b19d70aa71abaccf74a1f/Mobile_Price_Classification.csv")

In [44]:
#input_data=pd.read_csv('Mobile_Price_Classification.csv')

In [45]:
input_data

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0.0,1,0,7,0.6,188.0,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1.0,0,1,53,0.7,136.0,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1.0,2,1,41,0.9,145.0,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0.0,0,0,10,0.8,131.0,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0.0,13,1,44,0.6,141.0,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,996,1700,1.0,1.9,0,0,1,54.0,0.5,170,...,17,644,913,2121,14,8,15,1,1,0
2996,997,609,0.0,1.8,1,0,0,13.0,0.9,186,...,2,1152,1632,1933,8,1,19,0,1,1
2997,998,1185,0.0,1.4,0,1,1,8.0,0.5,80,...,12,477,825,1223,5,0,14,1,0,0
2998,999,1533,1.0,0.5,1,0,0,50.0,0.4,171,...,12,38,832,2509,15,11,6,0,1,0


In [46]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(input_data,test_size=0.2)
train_data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
941,743,0,0.5,1.0,0,1,33,0.3,95.0,2,...,1587,1659,316,19,10,4,1,0,0,0
2890,891,1876,0.0,2.4,0,11,0,22.0,0.7,168,...,18,157,1512,1403,15,11,13,1,0,0
422,1503,1,0.7,0.0,10,1,43,0.2,166.0,4,...,841,1304,2122,15,1,2,1,0,0,2
2953,954,1053,0.0,2.5,1,12,1,3.0,0.6,86,...,19,910,1097,792,17,6,7,1,0,1
1517,911,0,1.4,1.0,3,0,14,0.7,199.0,4,...,1563,1692,2933,16,3,11,0,1,0,3


In [28]:
# Save training data to a csv file 
train_data.to_csv("./mobile_price_traindata.csv")

In [15]:
data_store = ws.get_default_datastore()
data_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-1896c5be-cb73-4775-bea9-388c806d3d0a",
  "account_name": "mlstrg141462",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [29]:
data_store.upload(src_dir='./',target_path='./')


Uploading an estimated of 11 files
Target already exists. Skipping upload for .amlignore
Target already exists. Skipping upload for .amlignore.amltmp
Target already exists. Skipping upload for automl.ipynb
Target already exists. Skipping upload for automl.ipynb.amltmp
Target already exists. Skipping upload for hyperparameter_tuning.ipynb
Target already exists. Skipping upload for hyperparameter_tuning.ipynb.amltmp
Target already exists. Skipping upload for Mobile_Price_Classification.csv
Target already exists. Skipping upload for mobile_price_traindata.csv
Target already exists. Skipping upload for .ipynb_aml_checkpoints/automl-checkpoint2021-2-27-17-36-48.ipynb
Target already exists. Skipping upload for .ipynb_aml_checkpoints/hyperparameter_tuning-checkpoint2021-2-27-17-37-29.ipynb
Target already exists. Skipping upload for .ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded 0 files


KeyboardInterrupt: 

In [ ]:
train_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './mobile_price_traindata.csv')])

In [39]:
train_data

,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
2465,983,1.0,1.6,0,7,1,55.0,0.4,168,8,8,205,963,1425,14,0,4,1,1,0
1735,1,0.5,0.0,11,1,48,0.1,121.0,8,20,606,1316,3484,11,10,16,1,0,0,3
1272,0,0.7,1.0,1,1,25,0.8,91.0,2,8,1097,1461,285,19,7,3,1,0,1,0
437,0,2.8,1.0,1,0,36,0.5,130.0,6,10,1452,1744,985,12,11,7,1,1,0,1
2536,740,0.0,3.0,1,0,1,47.0,0.7,166,3,5,417,596,3578,15,5,20,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,1,1.6,1.0,6,1,6,0.5,162.0,6,17,179,1559,3352,9,1,3,1,1,1,3
1016,1,2.8,0.0,0,1,54,0.1,172.0,7,15,169,1916,1414,6,1,19,1,0,1,0
1424,0,2.4,0.0,5,1,6,0.9,92.0,6,9,629,1299,1309,12,1,14,1,0,0,1
280,1,2.8,0.0,3,1,36,1.0,119.0,6,20,357,527,3576,17,16,9,1,0,0,3


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [47]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="price_range",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=5,
                             **automl_settings
)

In [48]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

ConfigException: ConfigException:
	Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service